In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def evalf(x):  
  #Input: x is a numpy array of size 3 
  assert type(x) is np.ndarray and len(x) == 3 #do not allow arbitrary arguments
  #arr=np.array([(1/(8^i))*((x[i-1]-2^i)^2) for i in range(1,4)])
  #return np.sum(arr)
  return (1/8)*((x[0]-2)**2) + (1/8**2)*((x[1]-2**2)**2) + (1/8**3)*((x[2]-2**3)**2)

In [3]:
def evalg(x):  
  #Input: x is a numpy array of size 3
  assert type(x) is np.ndarray and len(x) == 3 #do not allow arbitrary arguments 
  
  return np.array([(1/4)*(x[0]-2), (1/32)*(x[1]-4), (1/256)*(x[2]-8)])

In [4]:
def compute_steplength_exact(gradf, A):
  assert type(gradf) is np.ndarray and len(gradf) == 3 
  assert type(A) is np.ndarray and A.shape[0] == 3 and  A.shape[1] == 3 #allow only a 3x3 array
   
  a_1 =np.linalg.multi_dot([gradf,gradf])
  a_22=np.matmul(np.transpose(gradf),A)
  a_2 =np.linalg.multi_dot([a_22,gradf])
  step_length=a_1/(2*a_2)
  
  return step_length

In [5]:
def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): 
  assert type(x) is np.ndarray and len(x) == 3
  assert type(gradf) is np.ndarray and len(gradf) == 3 
  
  alpha = alpha_start
  pk=-gradf
  while evalf(x+alpha*pk)>evalf(x)+gamma*alpha*np.linalg.multi_dot([gradf,pk]):
    alpha=rho*alpha

  #print('final step length:',alpha)
  return alpha

In [6]:
EXACT_LINE_SEARCH = 1
BACKTRACKING_LINE_SEARCH = 2
CONSTANT_STEP_LENGTH = 3

In [7]:
def find_minimizer(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 3 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  # construct a suitable A matrix for the quadratic function 
  A = np.array([[1/8,0,0],[0,1/64,0],[0,0,1/512]])
  x = start_x
  g_x = evalg(x)

  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    #print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)

  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
  
    if line_search_type == EXACT_LINE_SEARCH:
      step_length = compute_steplength_exact(g_x, A) #call the new function you wrote to compute the steplength
      #raise ValueError('EXACT LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma) #call the new function you wrote to compute the steplength
      #raise ValueError('BACKTRACKING LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 0.1
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here   
    x = np.subtract(x, np.multiply(step_length,g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point

    #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x ,evalf(x), k

**Q 2**

In [8]:
my_start_x = np.array([1,1,1])
my_tol= 1e-5

print('Using Exact line search:')
x_els, obje, itere = find_minimizer(my_start_x, my_tol, EXACT_LINE_SEARCH)
print('The minimizer is: ',x_els)
print('The minimum function value is: ',obje)
print('Number of iterations taken is: ',itere)

print('Using Backtracking line search:')
x_bls, obj, iter = find_minimizer(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, 1, 0.5,0.5)
print('The minimizer is: ',x_bls)
print('The minimum function value is: ',obj)
print('Number of iterations taken is: ',iter)

Using Exact line search:
The minimizer is:  [2.00001649 4.         7.99768767]
The minimum function value is:  1.0477122031231117e-08
Number of iterations taken is:  151
Using Backtracking line search:
The minimizer is:  [2.         4.         7.99744063]
The minimum function value is:  1.2793697352715871e-08
Number of iterations taken is:  2022


**Q 3**

In [9]:
x_start=np.array([1/64,1/8,1])
tol=1e-10

In [10]:
print('For Exact line search:')
x_bls_1, obj_1, iter_1 = find_minimizer(x_start, tol, EXACT_LINE_SEARCH)
print('The minimizer is: ',x_bls_1)
print('The minimum function value is: ',obj_1)
print('Number of iterations taken is: ',iter_1)

For Exact line search:
The minimizer is:  [2.         4.         7.99999998]
The minimum function value is:  9.150071377581033e-19
Number of iterations taken is:  269


In [11]:
print('For Backtracking line search:')
x_bls_2, obj_2, iter_2 = find_minimizer(x_start, tol, BACKTRACKING_LINE_SEARCH, 1, 0.5,0.5)
print('The minimizer is: ',x_bls_2)
print('The minimum function value is: ',obj_2)
print('Number of iterations taken is: ',iter_2)

For Backtracking line search:
The minimizer is:  [2.         4.         7.99999997]
The minimum function value is:  1.2748574165464873e-18
Number of iterations taken is:  4964


Number of iterations taken in case of Exact line search method is far less than that in Backtracking line search case because the Exact line search is always most efficient(if possible to apply).

**Q 4**

In [12]:
def evalf_4(x):  
  #Input: x is a numpy array of size 4
  assert type(x) is np.ndarray and len(x) == 4 #do not allow arbitrary arguments
  #arr=np.array([(1/(8^i))*((x[i-1]-2^i)^2) for i in range(1,4)])
  #return np.sum(arr)
  return (1/8)*((x[0]-2)**2) + (1/8**2)*((x[1]-2**2)**2) + (1/8**3)*((x[2]-2**3)**2) + (1/8**4)*((x[3]-2**4)**2)

In [13]:
def evalg_4(x):  
  #Input: x is a numpy array of size 4
  assert type(x) is np.ndarray and len(x) == 4 #do not allow arbitrary arguments 
  
  return np.array([(1/8)*(x[0]-2), (1/32)*(x[1]-4), (1/256)*(x[2]-8), (1/2048)*(x[3]-16)])

In [14]:
def compute_steplength_exact_4(gradf, A):
  assert type(gradf) is np.ndarray and len(gradf) == 4
  assert type(A) is np.ndarray and A.shape[0] == 4 and  A.shape[1] == 4 #allow only a 4x4 array
   
  a_1 =np.linalg.multi_dot([gradf,gradf])
  a_22=np.matmul(np.transpose(gradf),A)
  a_2 =np.linalg.multi_dot([a_22,gradf])
  step_length=a_1/(2*a_2)
  
  return step_length

In [15]:
def compute_steplength_backtracking_4(x, gradf, alpha_start, rho, gamma): 
  assert type(x) is np.ndarray and len(x) == 4
  assert type(gradf) is np.ndarray and len(gradf) == 4 
  
  alpha = alpha_start
  pk=-gradf
  while evalf_4(x+alpha*pk)>evalf_4(x)+gamma*alpha*np.linalg.multi_dot([gradf,pk]):
    alpha=rho*alpha

  #print('final step length:',alpha)
  return alpha

In [16]:
def find_minimizer_4(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 4, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 4 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  # construct a suitable A matrix for the quadratic function
  A = np.array([[1/8,0,0,0],[0,1/64,0,0],[0,0,1/512,0],[0,0,0,1/4096]])
  x = start_x
  g_x = evalg_4(x)

  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]

  k = 0

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
  
    if line_search_type == EXACT_LINE_SEARCH:
      step_length = compute_steplength_exact_4(g_x, A) #call the new function you wrote to compute the steplength
    elif line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_4(x,g_x, alpha_start,rho, gamma) #call the new function you wrote to compute the steplength
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 0.1
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here   
    x = np.subtract(x, np.multiply(step_length,g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg_4(x) #compute gradient at new point

  return x ,evalf_4(x), k

In [17]:
x_start_4=np.array([1/512,1/64,1/8,1])
tol=1e-10

In [18]:
print('For Exact line search:')
x_bls_3, obj_3, iter_3 = find_minimizer_4(x_start_4, tol, EXACT_LINE_SEARCH)
print('The minimizer is: ',x_bls_3)
print('The minimum function value is: ',obj_3)
print('Number of iterations taken is: ',iter_3)

For Exact line search:
The minimizer is:  [ 2.          4.          8.         15.99999989]
The minimum function value is:  2.7116323920795455e-18
Number of iterations taken is:  295


In [19]:
print('For Backtracking line search:')
x_bls_4, obj_4, iter_4 = find_minimizer_4(x_start_4, tol, BACKTRACKING_LINE_SEARCH, 1, 0.5,0.5)
print('The minimizer is: ',x_bls_4)
print('The minimum function value is: ',obj_4)
print('Number of iterations taken is: ',iter_4)

For Backtracking line search:
The minimizer is:  [ 2.         4.         8.        15.9999998]
The minimum function value is:  1.0237544252113109e-17
Number of iterations taken is:  37079


Yes, Exact line search is taking less number of iterations as compared to Backtracking line search.

Here the Exact line search method is taking almost same number of iterations as in Exact line search of last ques. But the number of iterations in case of Backtracking line search has increased significantly(almost 10 times) with increase in N.

**Q 5**

I think for large values of N Backtracking line search method will take very large number of iterations as only increasing N from 3 to 4 resulted in almost 10 times increase in number of iterations. Hence exact line search method will be more efficient.